## IMPORTS

In [1]:
import pandas as pd
import numpy as np
import scipy as sc
import seaborn as sns
import math
from datetime import datetime as dt
import matplotlib as mt
import matplotlib.pyplot as plt
from matplotlib import pylab
from matplotlib import cm
import json
import regex as re
pd.options.mode.chained_assignment = None  # default='warn'
pd.options.display.max_columns = 500
plt.rcParams["figure.figsize"] = [12, 9]
plt.rcParams.update({'font.size': 18})
mt.rc('xtick', labelsize=12) 
mt.rc('ytick', labelsize=12) 

### Function (unused) to calculate the week # at the month level (looking for a better option because this is currently inaccurate

In [2]:
def week_of_month(dt):
    """ Returns the week of the month for the specified date.
    """
    first_day = dt.replace(day=1)
    dom = dt.day
    adjusted_dom = dom + first_day.weekday()
    
    return int(ceil(adjusted_dom/7.0))

### CONVERT STRINGS TO ENCODED DATETIMES AND USE THESE TO ACCESS THE ROOT DATE AND YEAR

In [3]:
data = pd.read_csv("full_meter_data.csv")
data["purchased"] = pd.to_datetime(data["purchased"])
data["expiry"] = pd.to_datetime(data["expiry"])

In [4]:
data["date"] = data["purchased"].dt.date
data["year"] = data["purchased"].dt.year
#data["week"] = data["purchased"].apply(week_of_month)

## MAPPING INDIVIDUAL STALLS TO PARKING LOTS
This provides both stall counts for each lot, as well as tells us which transactions correspond to each lot

In [5]:
stall_data = pd.read_csv("stall_inventory.csv")

In [6]:
location_dict = {}
stall_dict = {}
for i, row in stall_data.iterrows():
    # add location
    location = row.location
    if location in location_dict:
        location_dict[location] += row.num_stalls
    else:
        location_dict[location] = row.num_stalls
        # add stall
    for stall in range(row.start_num, row.end_num + 1):
        stall_dict[stall] = location

**A new DF column that allows us to access transactions in a more granular time-based context**

In [7]:
data["month_day_hr"] = list(zip(data.start_month, data.start_weekday, data.start_hr_12))

**Declaring two global variables (lots and context) that allow us to do a dictionary accumulation**

In [8]:
lots = list(data.lot.unique())

In [9]:
context = list(data.month_day_hr.unique())

In [21]:
# create map from 12hr to 24hr
hr_map = {'12am': 0, 
          '1am': 1, 
          '2am': 2, 
          '3am': 3, 
          '4am': 4, 
          '5am': 5, 
          '6am': 6, 
          '7am': 7, 
          '8am': 8, 
          '9am': 9, 
          '10am': 10, 
          '11am': 11, 
          '12pm': 12, 
          '1pm': 13,
          '2pm': 14, 
          '3pm': 15, 
          '4pm': 16, 
          '5pm': 17, 
          '6pm': 18, 
          '7pm': 19, 
          '8pm': 20, 
          '9pm': 21, 
          '10pm': 22, 
          '11pm': 23}

In [38]:
# create map from month and weekday to num days
months = list(data.start_month.unique())
days = list(data.start_weekday.unique())
num_day_dict = {}
for month in months:
    num_day_dict[month] = {}
    for day in days:
        num_day_dict[month][day] = len(list(data.query("(start_month == @month) & (start_weekday == @day)").date.unique()))

In [10]:
data.head(10)

,stall,purchased,expiry,lot,start_month,start_weekday,start_hr_24,start_hr_12,end_month,end_weekday,end_hr_24,end_hr_12,date,year,month_day_hr
0,1724,2015-01-01 09:39:15,2015-01-01 10:09:15,Bermuda,1,Thursday,9,9am,1,Thursday,10,10am,2015-01-01,2015,"(1, Thursday, 9am)"
1,3637,2015-01-01 10:49:53,2015-01-01 12:07:53,Lot 7--W. Troy center,1,Thursday,10,10am,1,Thursday,12,12pm,2015-01-01,2015,"(1, Thursday, 10am)"
2,5216,2015-01-01 11:51:25,2015-01-01 13:09:25,Lot 13--E. 9-Mile (Como's),1,Thursday,11,11am,1,Thursday,13,1pm,2015-01-01,2015,"(1, Thursday, 11am)"
3,1324,2015-01-01 12:05:12,2015-01-01 13:35:12,E. Troy,1,Thursday,12,12pm,1,Thursday,13,1pm,2015-01-01,2015,"(1, Thursday, 12pm)"
4,3948,2015-01-01 12:10:38,2015-01-01 13:40:38,Lot 10--Library,1,Thursday,12,12pm,1,Thursday,13,1pm,2015-01-01,2015,"(1, Thursday, 12pm)"
5,2381,2015-01-01 12:19:33,2015-01-01 15:19:33,Lot 3--S. Withington,1,Thursday,12,12pm,1,Thursday,15,3pm,2015-01-01,2015,"(1, Thursday, 12pm)"
6,1499,2015-01-01 12:31:04,2015-01-01 14:31:03,W. Nine Mile,1,Thursday,12,12pm,1,Thursday,14,2pm,2015-01-01,2015,"(1, Thursday, 12pm)"
7,2382,2015-01-01 12:41:44,2015-01-01 15:41:44,Lot 3--S. Withington,1,Thursday,12,12pm,1,Thursday,15,3pm,2015-01-01,2015,"(1, Thursday, 12pm)"
8,5218,2015-01-01 12:50:23,2015-01-01 14:50:23,Lot 13--E. 9-Mile (Como's),1,Thursday,12,12pm,1,Thursday,14,2pm,2015-01-01,2015,"(1, Thursday, 12pm)"
9,3948,2015-01-01 13:38:19,2015-01-01 14:56:19,Lot 10--Library,1,Thursday,13,1pm,1,Thursday,14,2pm,2015-01-01,2015,"(1, Thursday, 1pm)"


**EXPAND DATA FRAME**

In [11]:
# add start and end minute columns
def get_minute(dt):
    return(dt.minute)
data['start_min'] = data.purchased.apply(get_minute)
data['end_min'] = data.expiry.apply(get_minute)

In [12]:
data.head(3)

,stall,purchased,expiry,lot,start_month,start_weekday,start_hr_24,start_hr_12,end_month,end_weekday,end_hr_24,end_hr_12,date,year,month_day_hr,start_min,end_min
0,1724,2015-01-01 09:39:15,2015-01-01 10:09:15,Bermuda,1,Thursday,9,9am,1,Thursday,10,10am,2015-01-01,2015,"(1, Thursday, 9am)",39,9
1,3637,2015-01-01 10:49:53,2015-01-01 12:07:53,Lot 7--W. Troy center,1,Thursday,10,10am,1,Thursday,12,12pm,2015-01-01,2015,"(1, Thursday, 10am)",49,7
2,5216,2015-01-01 11:51:25,2015-01-01 13:09:25,Lot 13--E. 9-Mile (Como's),1,Thursday,11,11am,1,Thursday,13,1pm,2015-01-01,2015,"(1, Thursday, 11am)",51,9


In [29]:
# create dictionary to hold data
expand_dict = {}
def expand_row(row):
    row_counter = 0
    # hour of purchase
    expand_dict[str(row.name) + '.' + str(row_counter)] = {'stall': row.stall, 
                                                            'lot': row.lot, 
                                                            'year': row.year,
                                                            'month': row.start_month, 
                                                            'weekday': row.start_weekday, 
                                                            'hour': row.start_hr_24, 
                                                            'minutes': 60 - row.start_min}
    row_counter += 1
    # one day purchase
    if row.start_weekday == row.end_weekday:
        # add hours between hours of purchase and expiry
        for i in range(row.start_hr_24 + 1, row.end_hr_24):
            expand_dict[str(row.name) + '.' + str(row_counter)] = {'stall': row.stall, 
                                                                    'lot': row.lot, 
                                                                    'year': row.year,
                                                                    'month': row.start_month, 
                                                                    'weekday': row.start_weekday, 
                                                                    'hour': i, 
                                                                    'minutes': 60}
            row_counter += 1
    # multiple day purchase
    else:
        # first day
        for i in range(row.start_hr_24 + 1, 24):
            expand_dict[str(row.name) + '.' + str(row_counter)] = {'stall': row.stall, 
                                                                    'lot': row.lot, 
                                                                    'year': row.year,
                                                                    'month': row.start_month, 
                                                                    'weekday': row.start_weekday, 
                                                                    'hour': i, 
                                                                    'minutes': 60}
            row_counter += 1
        # middle day during three day purchase - only situation is Saturday - Monday
        # possible year is incorrect for next two loops - would happend when first day is Dec 31
        if row.start_weekday == 'Saturday' and row.end_weekday == 'Monday':
            for i in range(0, 24):
                expand_dict[str(row.name) + '.' + str(row_counter)] = {'stall': row.stall, 
                                                                        'lot': row.lot, 
                                                                        'year': row.year,
                                                                        'month': row.start_month, # possible this is incorrect
                                                                        'weekday': 'Sunday', 
                                                                        'hour': i, 
                                                                        'minutes': 60}
                row_counter += 1
        # last day
        for i in range(0, row.end_hr_24):
            expand_dict[str(row.name) + '.' + str(row_counter)] = {'stall': row.stall, 
                                                                    'lot': row.lot, 
                                                                    'year': row.year,
                                                                    'month': row.end_month, 
                                                                    'weekday': row.end_weekday, 
                                                                    'hour': i, 
                                                                    'minutes': 60}
            row_counter += 1
    # hour of expiry
    expand_dict[str(row.name) + '.' + str(row_counter)] = {'stall': row.stall, 
                                                            'lot': row.lot, 
                                                            'year': row.year,
                                                            'month': row.end_month, 
                                                            'weekday': row.end_weekday, 
                                                            'hour': row.end_hr_24, 
                                                            'minutes': row.end_min}

In [30]:
data.apply(expand_row, axis=1)

0          None
1          None
2          None
3          None
4          None
5          None
6          None
7          None
8          None
9          None
10         None
11         None
12         None
13         None
14         None
15         None
16         None
17         None
18         None
19         None
20         None
21         None
22         None
23         None
24         None
25         None
26         None
27         None
28         None
29         None
           ... 
3481402    None
3481403    None
3481404    None
3481405    None
3481406    None
3481407    None
3481408    None
3481409    None
3481410    None
3481411    None
3481412    None
3481413    None
3481414    None
3481415    None
3481416    None
3481417    None
3481418    None
3481419    None
3481420    None
3481421    None
3481422    None
3481423    None
3481424    None
3481425    None
3481426    None
3481427    None
3481428    None
3481429    None
3481430    None
3481431    None
Length: 3481432, dtype: 

In [33]:
expand_df = pd.DataFrame.from_dict(expand_dict, orient='index')
expand_df.head()

,stall,lot,year,month,weekday,hour,minutes
0.0,1724,Bermuda,2015,1,Thursday,9,21
0.1,1724,Bermuda,2015,1,Thursday,10,9
1.0,3637,Lot 7--W. Troy center,2015,1,Thursday,10,11
1.1,3637,Lot 7--W. Troy center,2015,1,Thursday,11,60
1.2,3637,Lot 7--W. Troy center,2015,1,Thursday,12,7


In [34]:
expand_df.to_csv("expanded_data.csv", index=False)

## **FORMING ESTIMATION DICTIONARIES (FULL DATASET + SPLIT OUT BY YEAR)**

In [11]:
# use expanded data frame
expand_df = pd.read_csv("expanded_data.csv")
expand_df.head()

,stall,lot,year,month,weekday,hour,minutes
0,1724,Bermuda,2015,1,Thursday,9,21
1,1724,Bermuda,2015,1,Thursday,10,9
2,3637,Lot 7--W. Troy center,2015,1,Thursday,10,11
3,3637,Lot 7--W. Troy center,2015,1,Thursday,11,60
4,3637,Lot 7--W. Troy center,2015,1,Thursday,12,7


**ALL DATA**

In [43]:
est_dict = {}
for lot in lots:
    lot_df = expand_df.query("lot == @lot")
    est_dict[lot] = {}
    for c in context:
        mon = c[0]
        day = c[1]
        hr = hr_map[c[2]]
        instance = lot_df.query("month == @mon & weekday == @day & hour == @hr")
        days = num_day_dict[mon][day]
        n = sum(instance.minutes)
        #print(instance)
        try:
            avg = (n/60)/days
            est_dict[lot][c] = avg/location_dict[lot]
        except:
            est_dict[lot][c] = 0

Bermuda
Lot 7--W. Troy center
Lot 13--E. 9-Mile (Como's)
E. Troy
Lot 10--Library
Lot 3--S. Withington
W. Nine Mile
Lot 8--W. Troy (Betty's)
NB Woodward
NONE
Vester
Lot 11--City Hall
Lot 1--W. Breckenridge
Lot 15--E. Breckenridge
Lot 2--N. Withington
Lot 4--Planavon
E. Nine Mile
Lot 9--Falvey
SB Woodward
W. Troy
Withington
W. Saratoga
E. Breckenridge
Planavon
E. Saratoga
Ardmoor


**2015**

In [187]:
est_dict_15 = {}
data_15 = data[data.year == 2015]
for lot in lots:
    lot_df = data_15[data_15.lot == lot]
    est_dict_15[lot] = {}
    for c in context:
        instance = lot_df[lot_df.month_day_hr == c]
        days = len(list(instance.date.unique()))
        n = len(instance)
        #print(instance)
        try:
            avg = n/days
            est_dict_15[lot][c] = avg/location_dict[lot]
        except:
            est_dict_15[lot][c] = 0

**2016**

In [208]:
est_dict_16 = {}
data_16 = data[data.year == 2016]
for lot in lots:
    lot_df = data_16[data_16.lot == lot]
    est_dict_16[lot] = {}
    for c in context:
        instance = lot_df[lot_df.month_day_hr == c]
        days = len(list(instance.date.unique()))
        n = len(instance)
        #print(instance)
        try:
            avg = n/days
            est_dict_16[lot][c] = avg/location_dict[lot]
        except:
            est_dict_16[lot][c] = 0

**2017**

In [218]:
est_dict_17 = {}
data_17 = data[data.year == 2017]
for lot in lots:
    lot_df = data_17[data_17.lot == lot]
    est_dict_17[lot] = {}
    for c in context:
        instance = lot_df[lot_df.month_day_hr == c]
        days = len(list(instance.date.unique()))
        n = len(instance)
        #print(instance)
        try:
            avg = n/days
            est_dict_17[lot][c] = avg/location_dict[lot]
        except:
            est_dict_17[lot][c] = 0

**2018**

In [219]:
est_dict_18 = {}
data_18 = data[data.year == 2018]
for lot in lots:
    lot_df = data_18[data_18.lot == lot]
    est_dict_18[lot] = {}
    for c in context:
        instance = lot_df[lot_df.month_day_hr == c]
        days = len(list(instance.date.unique()))
        n = len(instance)
        #print(instance)
        try:
            avg = n/days
            est_dict_18[lot][c] = avg/location_dict[lot]
        except:
            est_dict_18[lot][c] = 0

**2019**

In [220]:
est_dict_19 = {}
data_19 = data[data.year == 2019]
for lot in lots:
    lot_df = data_19[data_19.lot == lot]
    est_dict_19[lot] = {}
    for c in context:
        instance = lot_df[lot_df.month_day_hr == c]
        days = len(list(instance.date.unique()))
        n = len(instance)
        #print(instance)
        try:
            avg = n/days
            est_dict_19[lot][c] = avg/location_dict[lot]
        except:
            est_dict_19[lot][c] = 0

## **CREATING DATAFRAMES FROM THE ESTIMATION DICTIONARIES**

**ALL DATA**

In [45]:
est_df = pd.DataFrame.from_dict(data = est_dict).reset_index().drop(columns= ["NONE"]).rename(columns = {"level_0": "Month", "level_1": "Weekday", "level_2": "Time"})

**2015**

In [198]:
est_df_15 = pd.DataFrame.from_dict(data = est_dict_15).reset_index().drop(columns= ["NONE"]).rename(columns = {"level_0": "Month", "level_1": "Weekday", "level_2": "Time"})

**2016**

In [209]:
est_df_16 = pd.DataFrame.from_dict(data = est_dict_16).reset_index().drop(columns= ["NONE"]).rename(columns = {"level_0": "Month", "level_1": "Weekday", "level_2": "Time"})

**2017**

In [221]:
est_df_17 = pd.DataFrame.from_dict(data = est_dict_17).reset_index().drop(columns= ["NONE"]).rename(columns = {"level_0": "Month", "level_1": "Weekday", "level_2": "Time"})

**2018**

In [222]:
est_df_18 = pd.DataFrame.from_dict(data = est_dict_18).reset_index().drop(columns= ["NONE"]).rename(columns = {"level_0": "Month", "level_1": "Weekday", "level_2": "Time"})

**2019**

In [223]:
est_df_19 = pd.DataFrame.from_dict(data = est_dict_19).reset_index().drop(columns= ["NONE"]).rename(columns = {"level_0": "Month", "level_1": "Weekday", "level_2": "Time"})

### **SOME TRANSFORMATION FUNCTIONS FOR DF SORTING PURPOSES**

In [47]:
def hr_24(t):
    if "am" in t and "12" not in t:
        t = int(t[:-2])
    elif "pm" in t and "12" not in t:
        t = int(t[:-2]) + 12
    elif t == "12am":
        t = 0
    else:
        t = 12
    return t

In [48]:
day_dict = {"Monday": 0, "Tuesday": 1, "Wednesday": 2, "Thursday": 3, "Friday": 4, "Saturday": 5, "Sunday": 6}
def day_num(day, days = day_dict):
    return days[day]

### **APPLYING TRANSFORMATION FUNCTIONS**

**ALL DATA**

In [49]:
est_df['24_Time'] = est_df['Time'].apply(hr_24)


In [50]:
est_df["dayNum"] = est_df['Weekday'].apply(day_num)


**2015**

In [199]:
est_df_15['24_Time'] = est_df_15['Time'].apply(hr_24)


In [200]:
est_df_15["dayNum"] = est_df_15['Weekday'].apply(day_num)


**2016**

In [210]:
est_df_16['24_Time'] = est_df_16['Time'].apply(hr_24)


In [211]:
est_df_16["dayNum"] = est_df_16['Weekday'].apply(day_num)


**2017**

In [224]:
est_df_17['24_Time'] = est_df_17['Time'].apply(hr_24)


In [225]:
est_df_17["dayNum"] = est_df_17['Weekday'].apply(day_num)


**2018**

In [226]:
est_df_18['24_Time'] = est_df_18['Time'].apply(hr_24)


In [227]:
est_df_18["dayNum"] = est_df_18['Weekday'].apply(day_num)


**2019**

In [228]:
est_df_19['24_Time'] = est_df_19['Time'].apply(hr_24)


In [229]:
est_df_19["dayNum"] = est_df_19['Weekday'].apply(day_num)


## **SORTING - MOSTLY FOR VIEWABILITY DURING DEVELOPMENT AND CREATING CLEAN CSVs**

**ALL DATA**

In [51]:
est_df = est_df.sort_values(by=['Month', '24_Time', 'dayNum'])
est_df.to_csv("estimations.csv", index = False)

**2015**

In [201]:
est_df_15 = est_df_15.sort_values(by=['Month', '24_Time', 'dayNum'])
est_df_15.to_csv("estimations15.csv", index = False)

**2016**

In [215]:
est_df_16 = est_df_16.sort_values(by=['Month', '24_Time', 'dayNum'])
est_df_16.to_csv("estimations16.csv", index = False)

**2017**

In [230]:
est_df_17 = est_df_17.sort_values(by=['Month', '24_Time', 'dayNum'])
est_df_17.to_csv("estimations17.csv", index = False)

**2018**

In [231]:
est_df_18 = est_df_18.sort_values(by=['Month', '24_Time', 'dayNum'])
est_df_18.to_csv("estimations18.csv", index = False)

**2019**

In [232]:
est_df_19 = est_df_19.sort_values(by=['Month', '24_Time', 'dayNum'])
est_df_19.to_csv("estimations19.csv", index = False)

## **DEFINING GET_ESTIMATION FUNCTIONS AND TESTING THE DATAFRAME(S)**

In [52]:
#Input month number, time of day (in 12 hour format), and weekday (in string format) to receive
#a single row in the estimation DataFrame satisfying the condition - this will be important functionality
#for making calls to the REST API

def get_estimation(month, time, day, df = est_df):
    return df[(est_df.Month == month) & (est_df.Time == time) & (est_df.Weekday == day)]


In [61]:
get_estimation(7, "8pm", "Friday")

,Month,Weekday,Time,Bermuda,Lot 7--W. Troy center,Lot 13--E. 9-Mile (Como's),E. Troy,Lot 10--Library,Lot 3--S. Withington,W. Nine Mile,Lot 8--W. Troy (Betty's),NB Woodward,Vester,Lot 11--City Hall,Lot 1--W. Breckenridge,Lot 15--E. Breckenridge,Lot 2--N. Withington,Lot 4--Planavon,E. Nine Mile,Lot 9--Falvey,SB Woodward,W. Troy,Withington,W. Saratoga,E. Breckenridge,Planavon,E. Saratoga,Ardmoor,24_Time,dayNum
888,7,Friday,8pm,0.371605,0.979183,1.015097,0.669753,0.807911,1.042298,0.235717,1.102849,0.210031,0.784325,0.582813,0.533049,1.077603,0.725439,1.075278,0.301406,0.756038,0.171639,0.69,0.738386,0.207828,0.19881,0.261728,0.208951,0.042824,20,4


In [62]:
#Similar to get_estimation, but only returns the estimated occupancy for a single lot 
#rather than the entire row of the DataFrame, which contains the data for all the lots
def get_estimation_by_lot(month, time, day, lot, df = est_df):
    return list(df[(est_df.Month == month) & (est_df.Time == time) & (est_df.Weekday == day)][lot])[0]

In [63]:
get_estimation_by_lot(2, "5pm", "Friday", "Lot 3--S. Withington")

0.6250618374558303

In [239]:
get_estimation_by_lot(2, "5pm", "Friday", "Lot 3--S. Withington", df = est_df_15)

0.3630742049469965

In [240]:
get_estimation_by_lot(2, "5pm", "Friday", "Lot 3--S. Withington", df = est_df_16)

0.338339222614841

In [241]:
get_estimation_by_lot(2, "5pm", "Friday", "Lot 3--S. Withington", df = est_df_17)

0.5247349823321554

In [242]:
get_estimation_by_lot(2, "5pm", "Friday", "Lot 3--S. Withington", df = est_df_18)

0.42226148409893993

In [243]:
get_estimation_by_lot(2, "5pm", "Friday", "Lot 3--S. Withington", df = est_df_19)

0.36042402826855124